# Описание задания


Как было сказано на лекции, в этой домашней работе нужно разработать систему исправления опечаток. За полное задание можно получить 30 баллов. 15 баллов даются за компоненты вашей системы:

* Модель языка (2 балла)

* Модель ошибок (3 балла)

* Генератор исправлений с помощью нечеткого поиска в боре (4 балла)

* Классификатор (2 балла)

* Итерации (1 балл)

* Разные типы исправлений: словарные, split, join и раскладка (3 балла)

Остальные 15 баллов даются за качество вашей системы. По 5 баллов за качество по каждому пакету (Need no fix, Need fix, (Need split + Need join)/ 2). Четких критериев нет, поэтому эта оценка может меняться. Но примерные критерии следующие: выше 80% - 5 баллов, ниже 10% - 0 баллов.

Для создания модели языка и модели ошибок, в также для обучения классификатора можно использовать файл с запросами (скачать). В нем содержатся запросы и, через табуляцию, исправление (если есть).

Для проверки решения развернута система автоматической проверки. Чтобы ей воспользоваться, нужно сделать следующее:

1. Скрипт, исправляющий опечатки должен называться spellchecker.py

2. Он должен принимать на вход строки с запросами и для каждого возвращать его исправленный вариант. Обратите внимание, запрос может быть уже без ошибки, тогда должен вернуться оригинальный запрос.

3. Перед запуском spellchecker.py будут запускаться preinstall.sh и indexer.py. Первый скрипт позволяет вам настроить окружение. Второй можно использовать для создания модели языка и модели ошибок. В папке, где он запускается обязательно будет присутствовать файл queries_all.txt

4. Ваша система должна работать со скоростью не менее 10 запросов в секунду (если дольше, то будут сниматься баллы).

5. Все скрипты нужно обернуть в архив (tgz) и послать по адресу sfera.spellchecker@mail.ru

6. Через некоторое время вам придет ответ с указанием того, сколько правильных ответов дала ваша система. Проверяется одновременно и то, что запрос с ошибкой исправляется и то, что запрос без ошибки не исправляется. Самый простой скрипт, возвращающий введенный запрос, отработает со следующим результатом:
No need fix: 1000/1000 (100%)
Need fix: 0/1000 (0.00%)
Need join: 0/1000 (0.00%)
Need split: 0/1000 (0.00%)
Т.е. везде где не нужно исправлять, он ничего не исправил (и это хорошо), но везде, где нужно исправить, он ничего не исправил (и это плохо).
Вам нужно набрать как можно больше правильных ответов в каждой категории.

7. Письмо с итоговым результатом пишите с префиксом [FINAL]

Дедлайн: 13 мая

P.S. Если у вас остались вопросы по заданию, пишите их в комментариях к данному посту.

P.P.S: Указывайте в заголовке письма слово test, если хотите протестировать качество на небольшом объеме данных (т.к. если ваша программа работает медленно, то придется долго ждать пока пройдет полный набор тестов)

Внимание!
В письмах присылается STDERR вашего скрипта. Специально, чтобы вы могли понять, в чем ошибка. Использовать это для получения тестовых данных нечестно. Это будет караться баллами вплоть до незачета этой домашки. Тестировать свою программу вы можете на тех данных, которые у вас есть (в тесте данные аналогичные).

P.P.P.S: Понятно, что если вы пришлете скрипт, который просто выводит то, что прочитал, то у вас будет в No Need Fix 100%, а в других 0%. Очевидно, что 5 баллов за No Need Fix вы таким образом не получите. Оценка за No Need Fix не может быть выше, чем max(Need fix, Need Join, Need Split).

# Решение

## Загрузка данных

In [1]:
!cp drive/MyDrive/Informatics/Sphere@mail.ru/IR/hw_03/queries_all.txt.tar.gz queries_all.txt.tar.gz

In [2]:
!cp drive/MyDrive/Informatics/Sphere@mail.ru/IR/hw_03/queries_all.txt queries_all.txt

In [3]:
from tqdm.notebook import tqdm
import tarfile
import os

In [4]:
if not os.path.exists('queries_all.txt'):
    try:
        !cp drive/MyDrive/Informatics/Sphere@mail.ru/IR/hw_03/queries_all.txt queries_all.txt
    except Exception:
        !cp drive/MyDrive/Informatics/Sphere@mail.ru/IR/hw_03/queries_all.txt.tar.gz queries_all.txt.tar.gz
        with tarfile.open('queries_all.txt.tar.gz', 'r:gz') as ftar:
            ftar.extractall()
else:
    print('File exists')

File exists


In [157]:
import numpy as np

def get_score(correct_func, count=100, return_data=False, verbose=True):    
    wrong_cnt = 0  
    right_cnt = 0
    if return_data:
        wrong_positive = []
        wrong_negative = []
    
    wrong_indexes = np.random.randint(0, len(wrong), count)
    right_indexes = np.random.randint(0, len(right), count)
    
    for i in tqdm(list(range(count))):    
        
        idx = wrong_indexes[i]
        corrected = correct_func(wrong[idx])
        if corrected == right[idx]:
            wrong_cnt += 1
        else:
            if return_data:
                wrong_positive.append((wrong[idx], corrected, right[idx]))
            if verbose:
                print('WRONG:', wrong[idx])
                print('FIXED:', corrected)
                print('RIGHT:', right[idx])
                print()
            
        
        idx = right_indexes[i]
        corrected = correct_func(right[idx])
        if corrected == right[idx]:
            right_cnt += 1
        else:
            if return_data:
                wrong_negative.append((right[idx], corrected))
            if verbose:
                print('RIGHT:', right[idx])
                print('FIXED:', corrected)
                print()
            
            
    p = wrong_cnt / count + 1e-9  # True positive rate
    r = right_cnt / count + 1e-9  # True negative rate
    
    if return_data:
        wrong_negative.append(p)
        wrong_positive.append(r)        
        return 2 / (1/p + 1/r), wrong_negative, wrong_positive
    else:
        return 2 / (1/p + 1/r)

In [158]:
wrong, right = [], []

with open('queries_all.txt', 'rt') as f:
    file = f.read().split('\n')
    for line in tqdm(file):
        if '\t' in line:
            pair = line.split('\t')
            wrong.append(pair[0])
            right.append(pair[1])

In [159]:
!pip3 install python-Levenshtein
import Levenshtein

In [160]:
def dl_dist(s1, s2):
    dist_func = Levenshtein.distance
    return min(dist_func(s1, s2), dist_func(change_layout(s1), s2) + 1)

In [180]:
def search_by_func(req, n=5, dist_func=dl_dist, reverse=False, ans_only=True):
    all_up = req.isupper()
    upcases = []
    for c in req:
        upcases.append(c.isupper())
    req = req.lower()
    best_dists = []
    for word in language_model.words:
        dist = dist_func(word, req)
        best_dists.append((dist, word))
        best_dists.sort(reverse=reverse)
        if len(best_dists) > n:
            best_dists.pop()
    res = best_dists[0][1]
    if all_up:
        res = res.upper()
    else:
        for i in range(min(len(res), len(upcases))):
            if upcases[i]:
                res = res[:i] + res[i].upper() + res[i + 1:]
    return res if ans_only else best_dists

In [181]:
def correct_line(line):
    return ' '.join(search_by_func(word) for word in line.split())

In [185]:
'щиты' in language_model.words

True

In [183]:
%%time
get_score(correct_line, 10)

RIGHT: википедия  ген FLASH
FIXED: википедия ген FLASH

WRONG: кремло кроватт фото
FIXED: кремле кровати фото
RIGHT: кресло кровать фото

WRONG: деушк мостурбировает перед парнем
FIXED: двушку мастурбировает перед парнем
RIGHT: девушка мастурбировает перед парнем

RIGHT: оштукатуренные  фасады загородных домов фото
FIXED: оштукатуренные фасады загородных домов фото

WRONG: facebook log in
FIXED: facebook blog in
RIGHT: facebook login

WRONG: поск работы
FIXED: воск работы
RIGHT: поиск работы

RIGHT: тт66666668                                              999999999999999999999999999999999999999997\\ъъъъъъъъъъъъъъъъъъъъъъъъъъъъъъ============================="
FIXED: тт66666668 999999999999999999999999999999999999999997\\ъъъъъъъъъъъъъъъъъъъъъъъъъъъъъъ============================="

WRONG: смотреть узбецкий фильм марджона
FIXED: смотреть кузнецкий фильм марджона
RIGHT: смотреть узбекский фильм марджона

WRONG: как поставить приставки на передні стойки  на ланосі
FIXED: как поставить приста

0.4000000011111111

### Utils

In [171]:
import os
import re


def check_dir_or_create(directory):
    if not os.path.exists(directory):
        os.mkdir(directory)


def word_clean(word):
    return word
    res = re.sub('\W', '', word)
    return res or word


def get_clean_words(s):
    return list(map(word_clean, s.lower().split()))


layout = {'1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '0': '0',
          '-': '-', '=': '=', 'q': 'й', 'w': 'ц', 'e': 'у', 'r': 'к', 't': 'е', 'y': 'н', 'u': 'г', 'i': 'ш',
          'o': 'щ', 'p': 'з', '[': 'х', ']': 'ъ', '\\':'\\','a': 'ф', 's': 'ы', 'd': 'в', 'f': 'а', 'g': 'п',
          'h': 'р', 'j': 'о', 'k': 'л', 'l': 'д', ';': 'ж', '\'':'э', 'z': 'я', 'x': 'ч', 'c': 'с', 'v': 'м',
          'b': 'и', 'n': 'т', 'm': 'ь', ',': 'б', '.': 'ю', '/': '.', '!': '!', '@': '"', '#': '№', '$': ';',
          '%': '%', '^': ':', '&': '?', '*': '*', '(': '(', ')': ')', '_': '_', '+': '+', 'Q': 'Й', 'W': 'Ц',
          'E': 'У', 'R': 'К', 'T': 'Е', 'Y': 'Н', 'U': 'Г', 'I': 'Ш', 'O': 'Щ', 'P': 'З', '{': 'Х', '}': 'Ъ',
          '|': '/', 'A': 'Ф', 'S': 'Ы', 'D': 'В', 'F': 'А', 'G': 'П', 'H': 'Р', 'J': 'О', 'K': 'Л', 'L': 'Д',
          ':': 'Ж', '"': 'Э', 'Z': 'Я', 'X': 'Ч', 'C': 'С', 'V': 'М', 'B': 'И', 'N': 'Т', 'M': 'Ь', '<': 'Б',
          '>': 'Ю', '?': ',', ' ': ' ',
          
          '1': '1', '2': '2', '3': '3', '4': '4', '5': '5', '6': '6', '7': '7', '8': '8', '9': '9', '0': '0',
          '-': '-', '=': '=', 'й': 'q', 'ц': 'w', 'у': 'e', 'к': 'r', 'е': 't', 'н': 'y', 'г': 'u', 'ш': 'i',
          'щ': 'o', 'з': 'p', 'х': '[', 'ъ': ']', '\\':'\\','ф': 'a', 'ы': 's', 'в': 'd', 'а': 'f', 'п': 'g',
          'р': 'h', 'о': 'j', 'л': 'k', 'д': 'l', 'ж': ';', 'э': "'", 'я': 'z', 'ч': 'x', 'с': 'c', 'м': 'v',
          'и': 'b', 'т': 'n', 'ь': 'm', 'б': ',', 'ю': '.', '.': '/', '!': '!', '"': '@', '№': '#', ';': '$',
          '%': '%', ':': '^', '?': '&', '*': '*', '(': '(', ')': ')', '_': '_', '+': '+', 'Й': 'Q', 'Ц': 'W',
          'У': 'E', 'К': 'R', 'Е': 'T', 'Н': 'Y', 'Г': 'U', 'Ш': 'I', 'Щ': 'O', 'З': 'P', 'Х': '{', 'Ъ': '}',
          '/': '|', 'Ф': 'A', 'Ы': 'S', 'В': 'D', 'А': 'F', 'П': 'G', 'Р': 'H', 'О': 'J', 'Л': 'K', 'Д': 'L',
          'Ж': ':', 'Э': '"', 'Я': 'Z', 'Ч': 'X', 'С': 'C', 'М': 'V', 'И': 'B', 'Т': 'N', 'Ь': 'M', 'Б': '<',
          'Ю': '>', ',': '?', '`': 'ё', '~': 'Ё', 'ё': '`', 'Ё': '~'}


def change_layout(word):
    return ''.join([layout[elem] if elem in layout else '~' for elem in word])

### Error model

In [202]:
from collections import defaultdict
import json
import numpy as np
try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x

class ErrorModel:

    def __init__(self):      
        self.stat = defaultdict(lambda: defaultdict(int))
        # {'orig' : {'fix' : P(orig|fix)}}
        self._stat_size = 0
    
    @staticmethod
    def make_bigrams(word):
        word = '^' + word + '_'
        bigrams = []
        for i in range(len(word) - 1):
            bigrams.append(word[i:i + 2])
        return bigrams
    
    @staticmethod
    def _levenshtein_matrix(a, b):
        n, m = len(a), len(b)
        need_transpose = False
        if n > m:
            a, b = b, a
            n, m = m, n
            need_transpose = True

        current = list(range(n + 1))
        previous = current
        lv_matrix = np.array([current])
        for i in range(1, m + 1):
            prev_previous, previous, current = previous, current, [i] + [0] * n
            for j in range(1, n + 1):
                a_ne_b = int(a[j - 1] != b[i - 1])
                add    = previous[j] + 1
                delete = current[j - 1] + 1
                change = previous[j - 1] + a_ne_b
                
                # transpose = change
                # if j > 1 and i > 1 and a[j - 2] == b[i - 1] and a[j - 1] == b[j - 2]:
                #     transpose = prev_previous[j - 2] + a_ne_b

                current[j] = min(add, delete, change)#, transpose)
            lv_matrix = np.vstack((lv_matrix, [current]))
        return lv_matrix.T if need_transpose else lv_matrix
    
    def _add_stats(self, orig, fix):
        self.stat[orig][fix] += 1
        self._stat_size += 1

    def _get_stats(self, orig, fix):
        if orig in self.stat and fix in self.stat[orig]:
            return self.stat[orig][fix]
        return 1.0 / self._stat_size
    
    def _fill_stats(self, a, b, lv_matrix):
        i, j = len(a), len(b)
        cur_distance = lv_matrix[len(b), len(a)]

        while cur_distance != 0:
            add = lv_matrix[j - 1, i] if j > 0 else np.inf
            delete = lv_matrix[j, i - 1] if i > 0 else np.inf
            change = lv_matrix[j - 1, i - 1] if j > 0 and i > 0 else np.inf

            operation = np.argmin([change, add, delete]).item()

            if operation == 0:
                i -= 1
                j -= 1
                if cur_distance != change:
                    cur_distance = change
                    self._add_stats(a[i], b[j])
                
            elif operation == 1:
                j -= 1
                if cur_distance != add:
                    cur_distance = add
                    self._add_stats(a[i - 1][1] + '~', b[j])
            else:
                i -= 1
                if cur_distance != delete:
                    cur_distance = delete
                    self._add_stats(a[i], b[j - 1][1] + '~')
    
    def normalize_stat(self):
        for orig in self.stat:
            for fix in self.stat[orig]:
                self.stat[orig][fix] /= self._stat_size
    
    def make_model(self, filename="queries_all.txt"):
        self.stat = defaultdict(lambda: defaultdict(int))
        self._stat_size = 0
        with open(filename, 'r') as f:
            if filename == 'queries_all.txt':
                tqdm_cur = lambda x: tqdm(x, total=2000000)
            else:
                tqdm_cur = tqdm
            for line in tqdm_cur(f):
                if '\t' not in line:
                    continue
                wrong, right = line.lower().split('\t')
                wrong = get_clean_words(wrong)
                right = get_clean_words(right)
                
                if len(wrong) != len(right):  # join or split
                    continue

                for wrong_word, right_word in zip(wrong, right):
                    wrong_bigrams = self.make_bigrams(wrong_word)
                    right_bigrams = self.make_bigrams(right_word)
                    lv_matrix = self._levenshtein_matrix(wrong_bigrams, right_bigrams)
                    self._fill_stats(wrong_bigrams, right_bigrams, lv_matrix)

        self.normalize_stat()
    
    def save_model(self, filename='stats.json', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'w') as f:
            f.write(json.dumps((self._stat_size, self.stat)))
    
    def load_model(self, filename='stats.json', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'r') as f:
            self._stat_size, self.stat = json.loads(f.read())

In [209]:
from collections import defaultdict
import json
import numpy as np
try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x

class ErrorModel: # NO BIGRAM

    def __init__(self):      
        self.stat = defaultdict(lambda: defaultdict(int))
        # {'orig' : {'fix' : P(orig|fix)}}
        self._stat_size = 0

        self.alpha = 0.5
    
    def _add_stats(self, orig, fix):
        self.stat[orig][fix] = self.alpha ** (-dl_dist(orig, fix))
        self._stat_size += 1

    def _get_stats(self, orig, fix):
        if orig in self.stat and fix in self.stat[orig]:
            return self.stat[orig][fix]
        return self.alpha ** (-dl_dist(orig, fix))
    
    def make_model(self, filename="queries_all.txt"):
        self.stat = defaultdict(lambda: defaultdict(int))
        self._stat_size = 0
        with open(filename, 'r') as f:
            if filename == 'queries_all.txt':
                tqdm_cur = lambda x: tqdm(x, total=2000000)
            else:
                tqdm_cur = tqdm
            for line in tqdm_cur(f):
                if '\t' not in line:
                    continue
                wrong, right = line.lower().split('\t')
                wrong = get_clean_words(wrong)
                right = get_clean_words(right)
                
                if len(wrong) != len(right):  # join or split
                    continue

                for wrong_word, right_word in zip(wrong, right):
                    self._add_stats(wrong_word, right_word)
    
    def save_model(self, filename='stats.json', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'w') as f:
            f.write(json.dumps((self._stat_size, self.stat)))
    
    def load_model(self, filename='stats.json', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'r') as f:
            self._stat_size, self.stat = json.loads(f.read())

In [210]:
%%time
try:
    del error_model
except Exception:
    pass
error_model = ErrorModel()
error_model.make_model()
error_model.save_model()
del error_model
error_model = ErrorModel()
error_model.load_model()


CPU times: user 8.32 s, sys: 388 ms, total: 8.71 s
Wall time: 8.7 s


### Language model

In [192]:
import json
from collections import defaultdict

try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x

class LanguageModel:

    def __init__(self):
        self._init_by_saved_data()

    def _make_data_to_save(self):
        data = {
            'words': list(self.words),
            'unigram_words_count': self.unigram_words_count,
            'unigram_model': dict(self.unigram_model),
            'bigram_words_count': self.bigram_words_count,
            'bigram_model': dict(self.bigram_model),
        }
        return data
    
    def _init_by_saved_data(self, data=None):
        if data is None:
            self.words = set()
            self.unigram_words_count = 0
            self.unigram_model = defaultdict(int)
            # {'word': P(word)}

            self.bigram_words_count = 0
            self.bigram_model = defaultdict(lambda: defaultdict(int))
            # {'word_1': {'word_2': P(word_1|word_2)}}
            return
        
        self.words = set(data['words'])
        self.unigram_words_count = data['unigram_words_count']
        self.unigram_model = defaultdict(int, data['unigram_model'])
        self.bigram_words_count = data['bigram_words_count']
        self.bigram_model = defaultdict(lambda: defaultdict(int), data['bigram_model'])
    
    def normalize_stat(self):
        for word in self.unigram_model:
            self.unigram_model[word] /= self.unigram_words_count
        
        for word_1 in self.bigram_model:
            for word_2 in self.bigram_model[word_1]:
                self.bigram_model[word_1][word_2] /= self.bigram_words_count
    
    def make_model(self, filename="queries_all.txt"):
        self._init_by_saved_data()
        with open(filename, 'r') as f:
            if filename == 'queries_all.txt':
                tqdm_cur = lambda x: tqdm(x, total=2000000)
            else:
                tqdm_cur = tqdm
            for line in tqdm_cur(f):
                if '\t' not in line:
                    continue
                line = line.split('\t')[1].lower()

                words = get_clean_words(line)
                len_words = len(words)
                self.words.update(set(words))

                for i, word in enumerate(words):
                    self.unigram_words_count += 1
                    self.unigram_model[word] += 1

                    if i != len_words - 1:
                        self.bigram_words_count += 1
                        word_2 = words[i + 1]
                        self.bigram_model[word][word_2] += 1
        
        self.normalize_stat()
    
    def unigram_prob(self, word):
        if word in self.unigram_model:
            return self.unigram_model[word]
        return 1 / self.unigram_words_count
    
    def bigram_prob(self, word_1, word_2):
        if word_1 in self.bigram_model and word_2 in self.bigram_model[word_1]:
            return self.bigram_model[word_1][word_2]
        return 1 / self.bigram_words_count
    
    def query_prob(self, query):
        words = get_clean_words(query.lower())
        len_words = len(words)
        prob = 1
        for i, word in enumerate(words):
            if i != len_words - 1:
                word_2 = words[i + 1]
                prob *= 1 / self.bigram_prob(word, word_2)
            else:
                prob *= 1 / self.unigram_prob(word)
        return prob
    
    def predict_next_word(self, word, n=None):
        word = word.lower()
        if word not in self.bigram_model:
            return []
        all_words = list(self.bigram_model[word].items())
        all_words.sort(key=lambda x: x[1], reverse=True)
        result = [elem[0] for elem in all_words]
        if n:
            result = result[:n]
        return result
    
    def save_model(self, filename='language_model.json', directory='prepared_data'):
        check_dir_or_create(directory)
        data_to_save = self._make_data_to_save()
        with open(directory + '/' + filename, 'w') as f:
            f.write(json.dumps(data_to_save))
    
    def load_model(self, filename='language_model.json', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'r') as f:
            data = json.loads(f.read())
        self._init_by_saved_data(data)

In [193]:
%%time
try:
    del language_model
except Exception:
    pass
language_model = LanguageModel()
language_model.make_model()
language_model.save_model()
del language_model
language_model = LanguageModel()
language_model.load_model()


CPU times: user 6.05 s, sys: 274 ms, total: 6.32 s
Wall time: 6.34 s


### Дерево поиска

In [204]:
import pickle
try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x
import sys
sys.setrecursionlimit(1000000)

class Node:
    def __init__(self, symb):
        self.symb = symb
        self._next = {}

        self._is_end = False

    def add_word(self, word):
        if word[0] not in self._next:
            self._next[word[0]] = Node(word[0])
        if len(word) > 1:
            self._next[word[0]].add_word(word[1:])
        else:
            self._next[word[0]]._is_end = True


class SearchTree:
    def __init__(self, error_model, words=None):
        self.root = Node('~')
        self.error_model = error_model
        if words is not None:
            self.make_tree(words)

    def make_tree(self, words):
        self._next = {}
        for word in tqdm(words):
            if word:
                self.root.add_word(word)
    
    def search(self, word, max_error=10000, max_insert=10, max_delete=10, n=20):
        # TODO:
        # variants size limit
        # max_error make bigger if nothing found

        variants = {}
        
        def recursion_search(node, word, prefix='', 
                             current_error=0, current_insert=0, current_delete=0):
            if current_error > max_error:
                return
            if not word:
                if node._is_end:
                    last = variants.get(prefix)
                    last = last or np.inf
                    variants[prefix] = min(last, current_error)
                return

            if word[0] in node._next:
                recursion_search(
                    node._next[word[0]],
                    word[1:],
                    prefix + word[0],
                    current_error,
                    current_insert,
                    current_delete
                )

            if node.symb + word[0] in self.error_model.stat:
                if current_delete < max_delete and node.symb + '~' in self.error_model.stat[node.symb + word[0]]:
                    err = self.error_model.stat[node.symb + word[0]][node.symb + '~']
                    recursion_search(
                        node, 
                        word[1:], 
                        prefix, 
                        current_error + 1 / err,
                        current_insert,
                        current_delete + 1
                    )

                for next_symb in node._next:
                    if node.symb + next_symb in self.error_model.stat[node.symb + word[0]]:
                        err = self.error_model.stat[node.symb + word[0]][node.symb + next_symb]
                        recursion_search(
                            node._next[next_symb], 
                            word[1:], 
                            prefix + next_symb, 
                            current_error + 1 / err,
                            current_insert,
                            current_delete
                        )
            
            if current_insert < max_insert and node.symb + '~' in self.error_model.stat:
                for next_symb in node._next:
                    if node.symb + next_symb in self.error_model.stat[node.symb + '~']:
                        err = self.error_model.stat[node.symb + '~'][node.symb + next_symb]
                        # print(prefix, node.symb, next_symb, err)
                        recursion_search(
                            node._next[next_symb],
                            next_symb + word[1:],
                            prefix + next_symb,
                            current_error + 1 / err,
                            current_insert + 1,
                            current_delete
                        )

        recursion_search(self.root, word)
        translate_word = change_layout(word)
        recursion_search(self.root, translate_word)

        variants = list(variants.items())

        variants.sort(key=lambda x: x[1])
        variants = variants or [(word, 0)]
        result = [elem[0] for elem in variants]
        return result[:n]
    
    def save_model(self, filename='search_tree.pickle', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'wb') as f:
            pickle.dump(self.root, f)
    
    def load_model(self, filename='search_tree.pickle', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'rb') as f:
            self.root = pickle.load(f)

In [246]:
import pickle
try:
    from tqdm.notebook import tqdm
except ImportError:
    tqdm = lambda x, *args, **kwargs: x
import sys
sys.setrecursionlimit(1000000)

class Node:
    def __init__(self, symb):
        self.symb = symb
        self._next = {}

        self._is_end = False

    def add_word(self, word):
        if word[0] not in self._next:
            self._next[word[0]] = Node(word[0])
        if len(word) > 1:
            self._next[word[0]].add_word(word[1:])
        else:
            self._next[word[0]]._is_end = True


class SearchTree: # NO BIGRAM
    def __init__(self, error_model, words=None):
        self.root = Node('~')
        self.error_model = error_model
        if words is not None:
            self.make_tree(words)

    def make_tree(self, words):
        self._next = {}
        for word in tqdm(words):
            if word:
                self.root.add_word(word)
    
    def search(self, word, max_error=1.5, max_insert=10, max_delete=10, n=20):
        # TODO:
        # variants size limit
        # max_error make bigger if nothing found

        variants = {}
        
        def recursion_search(node, word, prefix='', 
                             current_error=0, current_insert=0, current_delete=0):
            if current_error > max_error:
                return
            if not word:
                if node._is_end:
                    last = variants.get(prefix)
                    last = last or np.inf
                    variants[prefix] = min(last, current_error)
                return

            if word[0] in node._next:
                recursion_search(
                    node._next[word[0]],
                    word[1:],
                    prefix + word[0],
                    current_error,
                    current_insert,
                    current_delete
                )

            if current_delete < max_delete:
                err = self.error_model._get_stats(prefix + word[0], prefix)
                recursion_search(
                    node, 
                    word[1:], 
                    prefix, 
                    current_error + 1 / err,
                    current_insert,
                    current_delete + 1
                )

            for next_symb in node._next:
                err = self.error_model._get_stats(prefix + word[0], prefix + next_symb)
                recursion_search(
                    node._next[next_symb], 
                    word[1:], 
                    prefix + next_symb, 
                    current_error + 1 / err,
                    current_insert,
                    current_delete
                )
            
            if current_insert < max_insert:
                for next_symb in node._next:
                    err = self.error_model._get_stats(prefix, prefix + next_symb)
                    recursion_search(
                        node._next[next_symb],
                        next_symb + word[1:],
                        prefix + next_symb,
                        current_error + 1 / err,
                        current_insert + 1,
                        current_delete
                    )

        recursion_search(self.root, word)
        translate_word = change_layout(word)
        recursion_search(self.root, translate_word)

        variants = list(variants.items())

        variants.sort(key=lambda x: x[1])
        variants = variants or [(word, 0)]
        result = [elem[0] for elem in variants]
        return result[:n]
    
    def save_model(self, filename='search_tree.pickle', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'wb') as f:
            pickle.dump(self.root, f)
    
    def load_model(self, filename='search_tree.pickle', directory='prepared_data'):
        check_dir_or_create(directory)
        with open(directory + '/' + filename, 'rb') as f:
            self.root = pickle.load(f)

In [247]:
%%time
try:
    del tree
except Exception:
    pass
tree = SearchTree(error_model, language_model.words)
tree.save_model()
del tree
tree = SearchTree(error_model)
tree.load_model()


CPU times: user 15.2 s, sys: 466 ms, total: 15.6 s
Wall time: 15.6 s


In [248]:
!ls prepared_data -lh

total 57M
-rw-r--r-- 1 root root 27M May 14 09:49 language_model.json
-rw-r--r-- 1 root root 17M May 14 10:08 search_tree.pickle
-rw-r--r-- 1 root root 14M May 14 09:59 stats.json


In [249]:
for elem in tree.search('gbdtn'):
    print(elem)

пивот
пишет
живет
пиво
пива
пит
питт
писе
пилот
пирит
пират
пищит
пишут
писей
писес
пинает
пирей
питер
пищей
пишем


### Fixer

In [239]:
from itertools import product

class Fixer:

    def __init__(self, language_model, tree):
        self.language_model = language_model
        self.tree = tree
    
    def fix(self, query):
        query = query.lower()
        query_variants = []
        for word in query.split():
            query_variants.append(self.tree.search(word))
        
        query_variants = product(*query_variants)
        result_variants = []
        for var in tqdm(query_variants):
            var = ' '.join(var)
            prob = self.language_model.query_prob(var)
            result_variants.append((prob, var))
        
        result_variants.sort()
        return result_variants[0][1]

In [240]:
fixer = Fixer(language_model, tree)
fixer.fix('CVJNHTNM dblt')

'"смотреть виды'